# Test using top and bottom elevations polygons to determine image adjustment method

Rainey Aberle

October 2022

In [ ]:
import sys
import ee
import os
import rasterio as rio
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import glob
from shapely.geometry import Polygon, MultiPolygon, shape, Point, LineString

In [ ]:
# -----Path to snow-cover-mapping
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'

# -----Paths in directory
site_name = 'Wolverine'
# path to images
im_path = base_path + '../study-sites/' + site_name + '/imagery/PlanetScope/2016-2021/'
# path to AOI including the name of the shapefile
AOI_fn = base_path + '../../GIS_data/RGI_outlines/' + site_name + '_RGI.shp'
# path for output images
out_path = im_path + '../'
# path for output figures
figures_out_path = im_path + '../../../figures/'

# -----Set paths for output files
im_mosaic_path = out_path + 'mosaics/'
im_adj_path = out_path + 'adjusted-filtered/'
im_classified_path = out_path + 'classified/'

# -----Image file extensions (for mosaicing)
ext = 'SR_clip'

# -----Determine settings
plot_results = True # = True to plot figures of results for each image where applicable
skip_clipped = False # = True to skip images where bands appear "clipped", i.e. max blue SR < 0.8
crop_to_AOI = True # = True to crop images to AOI before calculating SCA
save_outputs = True # = True to save SCA images to file
save_figures = True # = True to save SCA output figures to file

# add path to functions
sys.path.insert(1, base_path+'functions/')
import ps_pipeline_utils as f

# Authenticate Google Earth Engine (GEE)
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()
    
# -----Load AOI as geopandas.GeoDataFrame
AOI = gpd.read_file(AOI_fn)

# -----Query GEE for DEM
DEM, AOI_UTM = f.query_GEE_for_DEM(AOI)

In [ ]:
def create_AOI_elev_polys(AOI, im_path, im_fns, DEM):
    '''
    Function to generate a polygon of the top 20th and bottom percentile elevations 
    within the defined Area of Interest (AOI).
    
    Parameters
    ----------
    AOI: geopandas.geodataframe.GeoDataFrame
        Area of interest used for masking images. Must be in same coordinate reference system (CRS) as the image
    im_path: str
        path in directory to the input images
    im_fns: list of str
        image file names located in im_path.
    DEM: xarray.DataSet
        digital elevation model
    
    Returns
    ----------
    polygons: list
        list of shapely.geometry.Polygons representing the top and bottom 20th percentiles of elevations in the AOI. 
        Median value in each polygon will be used to adjust images, depending on the difference. 
    im: xarray.DataArray
        image
    '''

    # -----Read one image that contains AOI to create polygon
    os.chdir(im_path)
    for i in range(0,len(im_fns)):
        # define image filename
        im_fn = im_fns[i]
        # open image
        im = rio.open(im_fn)
        # mask the image using AOI geometry
        mask = rio.features.geometry_mask(AOI.geometry,
                                       im.read(1).shape,
                                       im.transform,
                                       all_touched=False,
                                       invert=False)
        # check if any image values exist within AOI
        if (0 in mask.flatten()):
            break

    # -----Open image as xarray.DataArray
    im_rxr = rxr.open_rasterio(im_fn)
    # account for image scalar
    if np.nanmean(im_rxr.data[2]) > 1e3:
        im_rxr = im_rxr / 10000

    # -----Mask the DEM outside the AOI exterior
    mask_AOI = rio.features.geometry_mask(AOI.geometry,
                                  out_shape=(len(DEM.y), len(DEM.x)),
                                  transform=DEM.transform,
                                  invert=True)
    # convert maskto xarray DataArray
    mask_AOI = xr.DataArray(mask_AOI , dims=("y", "x"))
    # mask DEM values outside the AOI
    DEM_AOI = DEM.where(mask_AOI == True)

    # -----Interpolate DEM to the image coordinates
    band, x, y = im_rxr.indexes.values() # grab indices of image
    DEM_AOI_interp = DEM_AOI.interp(x=x, y=y, method="nearest") # interpolate DEM to image coordinates

    # -----Top elevations polygon
    # mask the bottom percentile of elevations in the DEM
    DEM_bottom_P = np.nanpercentile(DEM_AOI_interp.elevation.data.flatten(), 80)
    mask = xr.where(DEM_AOI_interp > DEM_bottom_P, 1, 0).elevation.data[0]
    # convert mask to polygon
    # adapted from: https://rocreguant.com/convert-a-mask-into-a-polygon-for-images-using-shapely-and-rasterio/1786/
    polygons_top = []
    for s, value in rio.features.shapes(mask.astype(np.int16), mask=(mask >0), transform=im.transform):
        polygons_top.append(shape(s))
    polygons_top = MultiPolygon(polygons_top)
    
    # -----Bottom elevations polygon
    # mask the top 80th percentile of elevations in the DEM
    DEM_bottom_P = np.nanpercentile(DEM_AOI_interp.elevation.data.flatten(), 20)
    mask = xr.where(DEM_AOI_interp < DEM_bottom_P, 1, 0).elevation.data[0]
    # convert mask to polygon
    # adapted from: https://rocreguant.com/convert-a-mask-into-a-polygon-for-images-using-shapely-and-rasterio/1786/
    polygons_bottom = []
    for s, value in rio.features.shapes(mask.astype(np.int16), mask=(mask >0), transform=im.transform):
        polygons_bottom.append(shape(s))
    polygons_bottom = MultiPolygon(polygons_bottom)
        
    return polygons_top, polygons_bottom, im_fn, im_rxr
    

In [ ]:
# -----Read image mosaic file names
os.chdir(im_mosaic_path)
im_mosaic_fns = glob.glob('*.tif')
im_mosaic_fns.sort()

# -----Create a polygon(s) of the top 20th percentile elevations within the AOI
polygon_top, polygon_bottom, im_fn, im = create_AOI_elev_polys(AOI_UTM, im_mosaic_path, im_mosaic_fns, DEM)
# plot
fig, ax = plt.subplots(figsize=(8,8))
ax.imshow(np.dstack([im.data[2], im.data[1], im.data[0]]), 
          extent=(np.min(im.x), np.max(im.x), np.min(im.y), np.max(im.y)))
AOI_UTM.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2, label='AOI')
for geom, count in list(zip(polygon_top.geoms, np.arange(0,len(polygon_top.geoms)))):
    xs, ys = geom.exterior.xy
    if count==0:
        ax.plot([x for x in xs], [y for y in ys], color='orange', label='top polygon(s)')
    else:
        ax.plot([x for x in xs], [y for y in ys], color='orange', label='_nolegend_')
for geom, count in list(zip(polygon_bottom.geoms, np.arange(0,len(polygon_bottom.geoms)))):
    xs, ys = geom.exterior.xy
    if count==0:
        ax.plot([x for x in xs], [y for y in ys], color='cyan', label='bottom polygon(s)')
    else:
        ax.plot([x for x in xs], [y for y in ys], color='cyan', label='_nolegend_')
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')
ax.set_title(im_fn)
fig.legend(loc='upper right')
fig.tight_layout()
plt.show()
    
# -----Loop through images
im_dts = []
SR_top_medians = np.zeros(len(im_mosaic_fns))
SR_bottom_medians = np.zeros(len(im_mosaic_fns))
differences = np.zeros(len(im_mosaic_fns))
for count, im_mosaic_fn in enumerate(im_mosaic_fns):
    
    print(im_mosaic_fn)
    
    im_dts = im_dts + [np.datetime64(str(im_mosaic_fn[0:4]) + '-' + str(im_mosaic_fn[4:6]) + '-' 
                      + str(im_mosaic_fn[6:8]) + 'T' + str(im_mosaic_fn[9:11]) + ':00')]
            
    # -----Load input image
    im_rxr = rxr.open_rasterio(im_mosaic_path + im_mosaic_fn)
    im_rio = rio.open(im_mosaic_path + im_mosaic_fn)
    # set no data values to NaN
    im_rxr = xr.where(im_rxr!=-9999)
    # account for image scalar multiplier if necessary
    im_scalar = 10000
    if np.nanmean(im_rxr.data[2]) > 1e3:
        im_rxr = im_rxr / im_scalar
    # define bands
    b = im_rxr.data[0]
    g = im_rxr.data[1]
    r = im_rxr.data[2]
    nir = im_rxr.data[3]
            
    # -----Return if image does not contain polygon
    # mask the image using polygon geometries
    mask_top = rio.features.geometry_mask([polygon_top],
                                   np.shape(b),
                                   im_rio.transform,
                                   all_touched=False,
                                   invert=False)
    mask_bottom = rio.features.geometry_mask([polygon_bottom],
                                   np.shape(b),
                                   im_rio.transform,
                                   all_touched=False,
                                   invert=False)
    # skip if image does not contain polygon
    if (0 not in mask_top.flatten()) or (0 not in mask_bottom.flatten()):
        print('image does not contain polygons... skipping.')
        continue 
            
    # -----Return if no real values exist within the SCA
    if (np.nanmean(b)==0) or (np.isnan(np.nanmean(b))):
        print('image does not contain any real values within the polygon... skipping.')
        continue
        
    # -----Filter image points outside the top polygon
    b_top_polygon = b[mask_top==0]
    g_top_polygon = g[mask_top==0]
    r_top_polygon = r[mask_top==0]
    nir_top_polygon = nir[mask_top==0]
    
    # -----Filter image points outside the bottom polygon
    b_bottom_polygon = b[mask_bottom==0]
    g_bottom_polygon = g[mask_bottom==0]
    r_bottom_polygon = r[mask_bottom==0]
    nir_bottom_polygon = nir[mask_bottom==0]
    
    # -----Calculate median value for each polygon and the mean difference between the two
    SR_top_median = np.mean([np.nanmedian(b_top_polygon), np.nanmedian(g_top_polygon),
                               np.nanmedian(r_top_polygon), np.nanmedian(nir_top_polygon)])
    SR_bottom_median = np.mean([np.nanmedian(b_bottom_polygon), np.nanmedian(g_bottom_polygon),
                               np.nanmedian(r_bottom_polygon), np.nanmedian(nir_bottom_polygon)])
    difference = np.mean([np.nanmedian(b_top_polygon) - np.nanmedian(b_bottom_polygon),
                            np.nanmedian(g_top_polygon) - np.nanmedian(g_bottom_polygon),
                            np.nanmedian(r_top_polygon) - np.nanmedian(r_bottom_polygon),
                            np.nanmedian(nir_top_polygon) - np.nanmedian(nir_bottom_polygon)])
    print('Mean value: Top=' + str(SR_top_median) + ', Bottom = ' + str(SR_bottom_median))
    print('Mean difference:'+str(difference))
    if (SR_top_median < 0.45) and (difference < 0.1):
        im_adj_method = 'ICE'
    else:
        im_adj_method = 'SNOW'
        
    differences[count] = difference
    SR_top_medians[count] = SR_top_median
    SR_bottom_medians[count] = SR_bottom_median
                   
    # -----Adjust SR 
    if im_adj_method=='SNOW':
        
        print('im_adj_method = SNOW')

        # define desired SR values at the bright area and darkest point for each band
        # bright area
        bright_b_adj = 0.94
        bright_g_adj = 0.95
        bright_r_adj = 0.94
        bright_nir_adj = 0.78
        # dark point
        dark_adj = 0.0
        
        # band_adjusted = band*A - B
        # A = (bright_adjusted - dark_adjusted) / (bright - dark)
        # B = (dark*bright_adjusted - bright*dark_adjusted) / (bright - dark)
        # blue band
        bright_b = np.nanmedian(b_top_polygon) # SR at bright point
        dark_b = np.nanmin(b) # SR at darkest point
        A = (bright_b_adj - dark_adj) / (bright_b - dark_b)
        B = (dark_b*bright_b_adj - bright_b*dark_adj) / (bright_b - dark_b)
        b_adj = (b * A) - B
        b_adj = np.where(b==0, np.nan, b_adj) # replace no data values with nan
        # green band
        bright_g = np.nanmedian(g_top_polygon) # SR at bright point
        dark_g = np.nanmin(g) # SR at darkest point
        A = (bright_g_adj - dark_adj) / (bright_g - dark_g)
        B = (dark_g*bright_g_adj - bright_g*dark_adj) / (bright_g - dark_g)
        g_adj = (g * A) - B
        g_adj = np.where(g==0, np.nan, g_adj) # replace no data values with nan
        # red band
        bright_r = np.nanmedian(r_top_polygon) # SR at bright point
        dark_r = np.nanmin(r) # SR at darkest point
        A = (bright_r_adj - dark_adj) / (bright_r - dark_r)
        B = (dark_r*bright_r_adj - bright_r*dark_adj) / (bright_r - dark_r)
        r_adj = (r * A) - B
        r_adj = np.where(r==0, np.nan, r_adj) # replace no data values with nan
        # nir band
        bright_nir = np.nanmedian(nir_top_polygon) # SR at bright point
        dark_nir = np.nanmin(nir) # SR at darkest point
        A = (bright_nir_adj - dark_adj) / (bright_nir - dark_nir)
        B = (dark_nir*bright_nir_adj - bright_nir*dark_adj) / (bright_nir - dark_nir)
        nir_adj = (nir * A) - B
        nir_adj = np.where(nir==0, np.nan, nir_adj) # replace no data values with nan
        
    elif im_adj_method=='ICE':
        
        print('im_adj_method = ICE')
        
        # define desired SR values at the bright area and darkest point for each band
        # bright area
        bright_b_adj = 0.58
        bright_g_adj = 0.59
        bright_r_adj = 0.57
        bright_nir_adj = 0.40
        # dark point
        dark_adj = 0.0
        
        # band_adjusted = band*A - B
        # A = (bright_adjusted - dark_adjusted) / (bright - dark)
        # B = (dark*bright_adjusted - bright*dark_adjusted) / (bright - dark)
        # blue band
        bright_b = np.nanmedian(b_top_polygon) # SR at bright point
        dark_b = np.nanmin(b) # SR at darkest point
        A = (bright_b_adj - dark_adj) / (bright_b - dark_b)
        B = (dark_b*bright_b_adj - bright_b*dark_adj) / (bright_b - dark_b)
        b_adj = (b * A) - B
        b_adj = np.where(b==0, np.nan, b_adj) # replace no data values with nan
        # green band
        bright_g = np.nanmedian(g_top_polygon) # SR at bright point
        dark_g = np.nanmin(g) # SR at darkest point
        A = (bright_g_adj - dark_adj) / (bright_g - dark_g)
        B = (dark_g*bright_g_adj - bright_g*dark_adj) / (bright_g - dark_g)
        g_adj = (g * A) - B
        g_adj = np.where(g==0, np.nan, g_adj) # replace no data values with nan
        # red band
        bright_r = np.nanmedian(r_top_polygon) # SR at bright point
        dark_r = np.nanmin(r) # SR at darkest point
        A = (bright_r_adj - dark_adj) / (bright_r - dark_r)
        B = (dark_r*bright_r_adj - bright_r*dark_adj) / (bright_r - dark_r)
        r_adj = (r * A) - B
        r_adj = np.where(r==0, np.nan, r_adj) # replace no data values with nan
        # nir band
        bright_nir = np.nanmedian(nir_top_polygon) # SR at bright point
        dark_nir = np.nanmin(nir) # SR at darkest point
        A = (bright_nir_adj - dark_adj) / (bright_nir - dark_nir)
        B = (dark_nir*bright_nir_adj - bright_nir*dark_adj) / (bright_nir - dark_nir)
        nir_adj = (nir * A) - B
        nir_adj = np.where(nir==0, np.nan, nir_adj) # replace no data values with nan
        
    # -----Plot results
    fig, ax = plt.subplots(1, 2, figsize=(8, 16))
    ax[0].imshow(np.dstack([r, g, b]), extent=(np.min(im_rxr.x.data)/1e3, np.max(im_rxr.x.data)/1e3, 
                                              np.min(im_rxr.y.data)/1e3, np.max(im_rxr.y.data)/1e3))
    ax[0].set_ylabel('Northing [km]')
    ax[0].set_xlabel('Easting [km]')
    ax[0].set_title('Raw image')
    ax[1].imshow(np.dstack([r_adj, g_adj, b_adj]), extent=(np.min(im_rxr.x.data)/1e3, np.max(im_rxr.x.data)/1e3, 
                                              np.min(im_rxr.y.data)/1e3, np.max(im_rxr.y.data)/1e3))
    ax[1].set_xlabel('Easting [km]')
    ax[1].set_title('Adjusted image')
    plt.show()
    
    print('-----')
    print(' ')


In [ ]:
from matplotlib.dates import DateFormatter

fig, ax = plt.subplots(figsize=(8,8))
lim = (np.datetime64('2016-05'), np.datetime64('2021-10'))
ax.plot(im_dts, differences, '.m', markersize=5)
ax.plot(im_dts, SR_top_medians, '.c', markersize=5)
ax.plot(im_dts, SR_bottom_medians, '.', color='orange', markersize=5)
date_form = DateFormatter("%y-%m")
ax.xaxis.set_major_formatter(date_form)
ax.grid()
plt.show()

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=['SR_top_median', 'SR_bottom_median', 'difference'])
df['SR_top_median'] = SR_top_medians
df['SR_bottom_median'] = SR_bottom_medians
df['difference'] = differences
df.index = im_dts

df.groupby(by=[df.index.year]).plot(marker='.')
plt.show()